# The full urgent care centre call model

**We can now update the model to include the final call centre logic!**

After a patient has spoken to a call operator their priority is triaged.  It is estimated that 40% of patients require a callback from a nurse.  There are 10 nurses available.  A nurse patient consultation has a Uniform distribution lasting between 10 and 20 minutes.

> ⏰ Some call centres run 24/7 while others are open for specified time window during the day.  So we need to cope with **both terminating and non-terminating systems**.

![model image](img/full_model.png "Urgent care call centre")

**Modifications needed**

* Add new default variables for the nurse consultation parameters
* Add new decision variables to `Experiment` for the no. nurses and the consultation distribution.
* Create a second `simpy.Resource` called `nurses` and add it to the simulation model.
* Create the nurse consultation process
* Modify the logic of `service` so that % of patients are called back.
* Collect results and estimate the waiting time for a nurse consultation and nurse utilisation.
* **Bonus:** Add an *optional* warm-up period event.

## 1. Standard Imports

In [1]:
import numpy as np
import pandas as pd
import simpy
import itertools

from abc import ABC, abstractmethod
from typing import Optional, Generator


In [2]:
## 2. Model imports

from model_utility import (
    Distribution,
    Exponential,
    Bernoulli,
    Uniform,
    Triangular,
    Experiment,
    WARM_UP_PERIOD,
    RESULTS_COLLECTION_PERIOD,
)

In [3]:
def trace(msg):
    """
    Turing printing of events on and off.

    Params:
    -------
    msg: str
        string to print to screen.
    """
    if TRACE:
        print(msg)

## 3. Modified model code

We will modify the model code and logic that we have already developed to include a nurse consultation for a proportion of the callers.  We create a new function called `nurse_consultation` that contains all the logic. We also need to modify the `service` function so that a proportion of calls are sent to the nurse consultation process.  

In [4]:
class SimplePathwayDelay(ABC):
    """
    Abstract base class representing a simple resource bound delay in a pathway 
    simulation using SimPy.
    
    This class encapsulates the logic for a service activity within a pathway,
    including resource request, service duration, and custom logic before 
    and after service.
    """
    
    def __init__(self, 
                 identifier: int,                 
                 env: simpy.Environment, 
                 resource: simpy.Resource,
                 delay_dist: Distribution,
                 args: Experiment,
                 trace_dp: Optional[int] = 2
    ) -> None:
        """
        Initialize the pathway delay.
        
        Parameters:
        ----------            
        identifier: int
            Unique identifier for this delay
            
        env: simpy.Environment
            The simulation environment in which the delay process
            is running.
            
        resource: simpy.resource
            The simpy resource to be requested

        delay_dist: Distribution 
            Used for sampling the process activity duration. Protocol pattern
            must implement the sample method
            
        args: Experiment 
            All model parameters

        trace_dp: int, optional (default=2)
            The number of decimal places that the time is displayed to when
            sending a message to trace.
        """
        self.identifier = identifier
        self.env = env
        self.resource = resource
        self.delay_dist = delay_dist
        self.args = args
        self.trace_dp = trace_dp
        self.waiting_time = -np.inf
        self.service_time = -np.inf
        self.total_time = -np.inf
            
    @abstractmethod
    def before_service_logic(self) -> None:
        """Simulation logic prior to the process entering a service activity"""
        pass

    @abstractmethod
    def after_service_logic(self) -> None:
        """Simulation logic after a service activity has been completed"""
        pass

    def service_logic(self) -> Generator:
        """
        Simulate the service process, including resource request and activity duration.
        
        Returns:
        --------
            A generator that can be yielded in a SimPy environment
        """
        # Record the start time for waiting
        start_wait = self.env.now
        
        # request a resource
        with self.resource.request() as req:
            yield req
    
            # record the waiting time for resource
            self.waiting_time = self.env.now - start_wait
                
            # sample the duration of the activity
            self.service_time = self.delay_dist.sample()
            
            # schedule process to begin again after activity duration
            yield self.env.timeout(self.service_time)
    
    def process(self) -> Generator:
        """
        Run the complete service process, including before and after hooks.
        
        Returns:
            A generator that can be yielded in a SimPy environment
        """
        start_time = self.env.now
        self.before_service_logic()
        yield from self.service_logic()
        self.after_service_logic()
        self.total_time = self.env.now - start_time
        
    def get_process(self) -> simpy.Process:
        """
        Get a SimPy process for this delay that can be started in the 
        environment.
        
        Returns:
        -------
            A SimPy process
        """
        return self.env.process(self.process())


In [5]:
class NurseConsultationPathway(SimplePathwayDelay):
    """
    Simulates the wait for a consultation with a nurse on the phone 🩺

    Inherits from SimplyPathwayDelay and hence follows this pattern:
    
    1. request and wait for a nurse resource
    2. phone consultation delay 
    3. release nurse and exit system
    """
    
    def __init__(
        self, 
        identifier: int, 
        env: simpy.Environment, 
        args: Experiment
    ) -> None:
        """
        NurseConsultationPathway

        Parameters:
        -----------        
        identifier: int
            Unique identifier for this delay
            
        env: simpy.Environment
            The simulation environment in which the delay process
            is running.
            
        args: Experiment 
            All model parameters
        """
        super().__init__(
            identifier=identifier,                 
            env=env,
            resource=args.nurses,
            delay_dist=args.nurse_dist,
            args=args)
        
        self.activity_name = "nurse consultation"

    def before_service(self) -> None:
        """Simulation logic prior to the process entering a service activity"""
        trace(
            f"{self.env.now:.{self.trace_dp}f}: Patient {self.identifier}"
            + f"waiting for nurse call back"
        )

    def after_service(self) -> None:
        """Simulation logic after service has completed."""
        # log metrics for process
        self.args.results["nurse_waiting_times"].append(self.waiting_time)
        self.args.results["total_nurse_call_duration"] += self.service_time

        trace(
            f"{self.env.now:.{self.trace_dp}f}: {self.activity_name}"
            + f" for {self.identifier} completed."
        )

In [6]:
class CallOperatorProcess(SimplePathwayDelay):
    """
    Encapsulates a Call Operator Process 💬
    
    This is a SimplePathwayDelay i.e. a FIFO resource bound delay.

    1. Request a call operator
    2. Simulate a delay while call operator triage takes place
    3. Release the resource
    4. Simulate the need for a nurse call back
    5. If needed launch an instance of NurseConsultationPathway
    """
    def __init__(
        self, 
        identifier: int, 
        env: simpy.Environment, 
        args: Experiment
    ) -> None:
        """
        CallOperatorProcess

        Parameters:
        -----------        
        identifier: int
            Unique identifier for this delay
            
        env: simpy.Environment
            The simulation environment in which the delay process
            is running.
            
        args: Experiment 
            All model parameters
        """
        
        super().__init__(
            identifier=identifier,                 
            env=env,
            resource=args.operators,
            delay_dist=args.call_dist,
            args=args)
        
        self.activity_name = "Call operator service"

    def before_service(self) -> None:
        """Simulation logic prior to the process entering a service activity"""
        trace(
            f"{self.env.now:.{self.trace_dp}f}: Patient {self.identifier}"
            + f" waiting for nurse call back."
        )

    def after_service(self) -> None:
        """Simulation logic after a service has completed.
        
        1. record process metrics
        2. simulate nurse call back process
        
        """
        # log metrics for process
        self.args.results["waiting_times"].append(self.waiting_time)
        self.args.results["total_call_duration"] += self.service_time

        trace(
            f"{self.env.now:.{self.trace_dp}f}: {self.activity_name}"
            + f" for {self.identifier} completed."
        )

        # nurse consultation?
        self.next_process()

    def next_process(self) -> None:
        """Simulate if a nurse call back is required."""
        
        callback_patient = self.args.callback_dist.sample()
        
        if callback_patient:
            nurse_pathway = NurseConsultationPathway(
                self.identifier, 
                self.env, 
                self.args
            )
            self.env.process(nurse_pathway.process())

In [7]:
class UrgentCallCentreModel:
    """
    Urgent care call centre model
    """
    def __init__(
        self,
        env: simpy.Environment, 
        args: Experiment
    ) -> None:
        """
        UrgentCallCentreModel

        Parameters:
        ------
        env: simpy.Environment
            The simpy environment for the simulation
    
        args: Experiment
            The settings and input parameters for the simulation.
        
        """
        self.env = env
        self.args = args

        self._init_resources()

    def _init_resources(self) -> None:
        """
        Init the number of resources
        and store in the arguments container object
        
        Resource list:
        1. call operators
        2. nurses
        """
        # create the environment object.
        self.args.operators = simpy.Resource(self.env, self.args.n_operators)
    
        # create the nurses resource
        self.args.nurses = simpy.Resource(self.env, self.args.n_nurses)

    def arrivals_generator(self) -> Generator:
        """
        📞 Call arrivals generator
        """
        # use itertools as it provides an infinite loop
        # with a counter variable that we can use for unique Ids
        for caller_count in itertools.count(start=1):
    
            # rhe sample distribution is defined by the experiment.
            inter_arrival_time = self.args.arrival_dist.sample()
            yield self.env.timeout(inter_arrival_time)
    
            trace(f"{self.env.now:.2f}: call arrives.")
    
            # create a service process
            operator_pathway = CallOperatorProcess(
                caller_count, 
                self.env, 
                self.args
            )

            # initialise pathway as simpy process
            self.env.process(operator_pathway.process())

    def warmup_complete(self, warm_up_period: float) -> Generator:
        """
        🥵 End of warm-up period Generator. 
        Generates a single event that occurs at the end of the warm-up period
        Used to reset results collection variables.
    
        Parameters:
        ----------
        warm_up_period: float
            Duration of warm-up period in simultion time units
        """
        yield self.env.timeout(warm_up_period)
        trace(f"{self.env.now:.2f}: Warm up complete.")

        # reset KPIs
        self.args.init_results_variables()

## 5. Model wrapper functions

Modifications to make to the `single_run` function:

1. Add a warm-up parameters called `wu_period`
1. Create and the nurses resource to the experiment
2. Schedule the `warm_up_complete` process.
3. After the simulation is complete calculate the mean waiting time and mean utilisation for nurses.

In [8]:
def single_run(
    experiment: Experiment, 
    rep: Optional[int] = 0,
    wu_period: Optional[float] = WARM_UP_PERIOD, 
    rc_period: Optional[float] = RESULTS_COLLECTION_PERIOD
) -> dict:
    """
    Perform a single run of the model and return the results

    Parameters:
    -----------

    experiment: Experiment
        The experiment/paramaters to use with model

    rep: int
        The replication number.

    wu_period: float, optional (default=WARM_UP_PERIOD)
        The initial transient period of the simulation
        Results from this period are removed from final computations.

    rc_period: float, optional (default=RESULTS_COLLECTION_PERIOD)
        The run length of the model following warm up where results are
        collected.
    """

    # results dictionary.  Each KPI is a new entry.
    run_results = {}

    # reset all results variables to zero and empty
    experiment.init_results_variables()

    # set random number set to the replication no.
    # this controls sampling for the run.
    experiment.set_random_no_set(rep)

    # environment is (re)created inside single run
    env = simpy.Environment()
    model = UrgentCallCentreModel(env, experiment)
    
    # we pass the experiment to the arrivals generator
    env.process(model.arrivals_generator())

    # add warm-up period event
    env.process(model.warmup_complete(wu_period))

    # run for warm-up + results collection period
    env.run(until=wu_period + rc_period)

    # end of run results: calculate mean waiting time
    run_results["01_mean_waiting_time"] = np.mean(
        experiment.results["waiting_times"]
    )

    # end of run results: calculate mean operator utilisation
    run_results["02_operator_util"] = (
        experiment.results["total_call_duration"]
        / (rc_period * experiment.n_operators)
    ) * 100.0

    # end of run results: nurse waiting time
    run_results["03_mean_nurse_waiting_time"] = np.mean(
        experiment.results["nurse_waiting_times"]
    )

    # end of run results: calculate mean nurse utilisation
    run_results["04_nurse_util"] = (
        experiment.results["total_nurse_call_duration"]
        / (rc_period * experiment.n_nurses)
    ) * 100.0

    # return the results from the run of the model
    return run_results

In [9]:
def multiple_replications(
    experiment: Experiment,
    wu_period: Optional[float] = WARM_UP_PERIOD, 
    rc_period: Optional[float] = RESULTS_COLLECTION_PERIOD,
    n_reps: Optional[int] = 5,
) -> pd.DataFrame:
    """
    Perform multiple replications of the model.

    Params:
    ------
    experiment: Experiment
        The experiment/paramaters to use with model

    rc_period: float, optional (default=DEFAULT_RESULTS_COLLECTION_PERIOD)
        results collection period.
        the number of minutes to run the model to collect results

    n_reps: int, optional (default=5)
        Number of independent replications to run.

    Returns:
    --------
    pandas.DataFrame
    """

    # loop over single run to generate results dicts in a python list.
    results = [
        single_run(experiment, rep, wu_period, rc_period) 
        for rep in range(n_reps)
    ]

    # format and return results in a dataframe
    df_results = pd.DataFrame(results)
    df_results.index = np.arange(1, len(df_results) + 1)
    df_results.index.name = "rep"
    return df_results

In [10]:
TRACE = False
scenario = Experiment(n_nurses=15, nurse_call_high=30.0)
results = multiple_replications(scenario, wu_period=50.0)
results.describe().round(1).T

,count,mean,std,min,25%,50%,75%,max
01_mean_waiting_time,5.0,3.2,0.9,1.8,3.0,3.2,3.9,4.0
02_operator_util,5.0,94.2,1.3,92.1,94.1,94.4,94.5,95.8
03_mean_nurse_waiting_time,5.0,3.0,1.7,1.6,1.6,2.0,4.7,4.9
04_nurse_util,5.0,88.3,3.6,83.7,86.9,87.9,89.5,93.6


In [11]:
TRACE = False
scenario = Experiment(n_nurses=15, nurse_call_high=30.0)
results = single_run(scenario, wu_period=50.0)
results

{'01_mean_waiting_time': 3.8923863988435095,
 '02_operator_util': 94.05731922168263,
 '03_mean_nurse_waiting_time': 4.945822358637601,
 '04_nurse_util': 93.57384685068052}